In [1]:
import sys
sys.path.append('..')
from utils import *
sys.path.append('../attacks/')
from cw import *
from torch import nn,norm
import torch
from torchvision import transforms, datasets
import numpy as np
from tqdm import tqdm
import cv2
import pandas as pd
import numpy as np
sys.path.append('../models/')
from alexnet import *
from densenet_201 import *
from mobilenet_v2 import *
from resnet_50 import *
from vgg_19 import *

In [2]:
imgs = datasets.ImageFolder('/home/hancy/dataset/imagenet3000/',transform=transforms.ToTensor())

In [3]:
wm_origin = cv2.imread('../img/logo.jpg')
wm_origin = cv2.cvtColor(wm_origin,cv2.COLOR_BGR2RGB)
wm_origin = transforms.ToTensor()(wm_origin).cuda()

In [10]:
def check_result(filename,direct_mode=False):
    cols = ['index','label','pred_label','perd_label','wm_l2','img_l2','alpha','beta']
    if(direct_mode):
        cols = ['index','label','pred_label','perd_label','wm_l2','img_l2']
    res = pd.read_csv(filename,names=cols,header=None)
    print((res['label'] == res['pred_label']).sum() / res['index'].count())
    print((res['label'] == res['perd_label']).sum() / res['index'].count())
    print(res['wm_l2'].sum() / res['wm_l2'].count())
    print(res['img_l2'].sum() / res['img_l2'].count())

In [11]:
def attack(filename: str,model: nn.Module, imgs:datasets, wm_origin: Tensor):
    block_size = 8
    alpha = 0.1
    c = 1
    kappa = 0
    steps = 10
    lr = 0.01
    with open(filename,'a') as file:
        for i in tqdm(range(len(imgs)), desc='Processing'):
            img = imgs[i][0].cuda()
            img = addborder(img,block_size)
            wm = transforms.Resize(img.size()[-2:])(wm_origin)
            target = torch.tensor([imgs[i][1]]).cuda()
            pred_label = model(img.unsqueeze(0)).argmax().item()
            perd_img = cw_direct(img,target,wm,alpha,block_size,model,c,kappa,steps,lr)
            wm_extracted = extract_wm(img,perd_img,alpha,block_size)
            res = model(perd_img.unsqueeze(0))
            perd_label = res.argmax().item()
            wm_l2_norm = norm(wm_extracted-wm).item()
            img_l2_norm = norm(perd_img-img).item()
            file.write('{},{},{},{},{},{}\n'.format(i,imgs[i][1],pred_label,perd_label,wm_l2_norm,img_l2_norm))

In [14]:
model = alexnet_IN().cuda().eval()

In [15]:
filename = '/home/hancy/code/adv_wm/atk_result/10161700_alexnet_cw_direct_imnet.txt'
attack(filename,model,imgs,wm_origin)
check_result(filename,True)

Processing:  75%|███████▍  | 2239/3000 [04:43<01:22,  9.26it/s]